In [1]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.retrievers import AzureCognitiveSearchRetriever
from langchain.llms import AzureOpenAI
from langchain.chains import LLMChain
from langchain.tools import BaseTool
from langchain.agents import ZeroShotAgent, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.memory.buffer_window import ConversationBufferWindowMemory
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from functools import reduce
import os

os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-03-15-preview'
os.environ['OPENAI_API_BASE'] = 'https://openai-nois-intern.openai.azure.com/'
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = "search-service01"
os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] = "public-data-index-alt"
os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = "73Swa5YqUR5IRMwUIqOH6ww2YBm3SveLv7rDmZVXtIAzSeBjEQe9"

retriever_public = AzureCognitiveSearchRetriever(content_key='content')
print(retriever_public)

# os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] = "public-index-ver2"
# retriever_private = AzureCognitiveSearchRetriever(content_key='content')
# print(retriever_private)

service_name='search-service01' index_name='public-data-index-alt' api_key='73Swa5YqUR5IRMwUIqOH6ww2YBm3SveLv7rDmZVXtIAzSeBjEQe9' api_version='2020-06-30' aiosession=None content_key='content'


In [2]:
# Load up your LLM
llm = AzureOpenAI(
    deployment_name="text-davinci-003",
    model_name="text-davinci-003",
    max_tokens=300,
    temperature=0
)

output_parser = CommaSeparatedListOutputParser()
history = []

def get_history_as_txt():
    txt = ""
    for i in history:
        txt += f"\nHuman: {i['user']}\n"
        txt += f"AI:{i['AI']}"
        
    return txt

def add_to_history(user_msg, ai_msg):
    history.append({'user': user_msg, 'AI': ai_msg})

In [3]:
keyword_template = """Given a sentence, extract maximum 2 most important sets of keywords and translate them to Vietnamese if they're in English and vice versa.
Your output will be in both languages if possible, separated by commas.
######
SENTENCE: What are New Ocean company's products ?
OUTPUT: New Ocean products, New Ocean sản phẩm
SENTENCE: Was Nestle a customer of New Ocean ?
OUTPUT: Nestle
SENTENCE: When was New Ocean founded ?
OUTPUT: New Ocean founded, New Ocean thành lập
SENTENCE: Các sản phẩm của công ty New Ocean là gì ?
OUTPUT: New Ocean sản phẩm, New Ocean products
SENTENCE: Ai chưa đóng tiền nước ?
OUTPUT: tiền nước, water bill
SENTENCE: {question}
OUTPUT:"""

keyword_templ = """Given a sentence, extract keywords and translate them to Vietnamese if they're in English and vice versa.
Your output will be in both languages if possible, separated by commas.
DON'T duplicate keywords.
######
EXAMPLE
SENTENCE: Ai chưa đóng tiền nước?
OUTPUT: tiền nước, water bill
######
SENTENCE: {output}
OUTPUT:"""

template = """Given a list of documents from a company called New Ocean, answer the question.
If the user asks any questions not related to New Ocean or NOIS, apologize and request another question from the user.
If the user greets you, respond accordingly. If you don't know the answer, just say that you don't know. Don't try to make up an answer. DON'T use other sources than the ones provided.
######
{summaries}
######{context}
Human: {question}
AI:"""

templ = PromptTemplate(
    input_variables = ['summaries', 'question', 'context'],
    template = template
)

In [4]:
past_prompt = """"Below is a history of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base about company information.
Generate a search query based on the conversation and the new question.

Chat history:{history}

Question:
{question}

Search query:
"""

In [16]:
query = "Who are the technical managers ?"
result = llm(past_prompt.format(history=get_history_as_txt(), question=query))
result

'"Technical managers" New Ocean Information System'

In [4]:
qaChain = load_qa_with_sources_chain(llm=llm, chain_type="stuff", prompt=templ)
keyword_getter = LLMChain(llm=llm, prompt=PromptTemplate.from_template(keyword_templ))

In [9]:
keywords = keyword_getter(query)['text']
keywords_lst = output_parser.parse(keywords)
print(keywords)
print(keywords_lst)

 NOIS, Microsoft, đối tác, partner
['NOIS', 'Microsoft', 'đối tác', 'partner']


In [5]:
# def get_docs_using_keyword_list(keywords, ret):
#     doc = []
#     keywords_lst = output_parser.parse(keywords)
    
#     num = int(4 / len(keywords_lst))
    
#     for i in keywords_lst:
#         doc.extend(ret.get_relevant_documents(i)[:num])
        
#     return doc

def get_docs_using_keyword_string(keywords, ret):
    return ret.get_relevant_documents(keywords)[:4]

In [6]:
query = "Is NOIS a Microsoft partner ?"
keywords = keyword_getter(query)['text']

doc = get_docs_using_keyword_string(keywords, retriever_public)

num = 1
for x in doc:
    x.metadata['source'] = f"doc-{num}"

    num += 1

result = qaChain({'input_documents': doc, 'question': query, 'context': get_history_as_txt()}, return_only_outputs=False)
    
add_to_history(query, result['output_text'])

In [7]:
result['output_text']

' Yes, New Ocean Information System (NOIS) is a Microsoft partner. They have been a partner since 2015 and have achieved Gold Application Integration, Gold Data Analytics, Gold Application Developments, and Gold Cloud Platform status in 2021.'

In [8]:
print(doc)

[Document(page_content=', \xa0 New Ocean IS understands our customers’ satisfaction is the key to our business so that we always strive for quality, extra values on our software delivery to customers.\r\nLearn More\r\n/* Title */\r\nTHE HISTORY\r\nOF\r\nNEW OCEAN INFORMATION SYSTEM\r\n/* line and circle */\r\n/* Milestone 2010 */\r\n2010\r\nFORMED AND STARTED OPERATION\r\nBe devired form IT Dept. of New Ocean\r\nManufacturing Solutions\r\n/* Milestone 2015 */\r\n2015\r\n/* content */\r\nbởi Microsoft.\r\nPartner\r\n/* Milestone 2017 */\r\n2017\r\n/* content */\r\nbởi Microsoft.\r\nPartner\r\nbởi Microsoft.\r\nSilver Cloud Platform\r\n/* Milestone 2018 */\r\n2018\r\n/* Milestone 2019 */\r\n2019\r\n/* content */\r\nbởi Microsoft.\r\nPartner\r\nbởi Microsoft.\r\nGold Application Development\r\nSilver Data Analytics\r\n/* Milestone 2020 */\r\n2020\r\n/* content */\r\nFASF is launched\r\n/* Milestone 2021 */\r\n2021\r\n/* content */\r\nbởi Microsoft.\r\nPartner\r\nbởi Microsoft.\r\nGold App

In [16]:
history = []

In [17]:
doc_txt = ""
for x in doc:
    doc_txt += x.page_content
    
context = get_history_as_txt()
    
msg = f"""Given a list of documents from a company called New Ocean, answer the question. When asked a question not related to New Ocean, say "Sorry, I will only answer questions related to New Ocean. Could you please ask another related question?".
If the user greets you, respond accordingly. If you don't know the answer, just say that you don't know. Don't try to make up an answer. DON'T use other sources than the ones provided.
######
{doc_txt}
######{context}"""

print(msg)
print(llm.get_num_tokens(msg))
print(f"Length of chat history: {llm.get_num_tokens(get_history_as_txt())}")

Given a list of documents from a company called New Ocean, answer the question. When asked a question not related to New Ocean, say "Sorry, I will only answer questions related to New Ocean. Could you please ask another related question?".
If the user greets you, respond accordingly. If you don't know the answer, just say that you don't know. Don't try to make up an answer. DON'T use other sources than the ones provided.
######

######
Human: What is NOIS ?
AI: New Ocean Information System (NOIS) là công ty có thâm niên hoạt động hơn 12 năm trong lĩnh vực cung cấp phần mềm, giải pháp công nghệ. NOIS nhanh chóng nắm bắt các xu hướng công nghệ mới và đưa sự phát triển của công nghệ và chuyển đổi số vào các lĩnh vực sản xuất một cách thông minh, đúng nơi, đúng việc.
Human: Who are the technical managers ?
AI: Các quản lý kỹ thuật của New Ocean bao gồm các chuyên gia công nghệ và các chuyên gia của các lĩnh vực liên quan. Họ có trách nhiệm quản lý các dự án công nghệ, đảm bảo rằng các dự á

In [11]:
# Define query tool for agent
class DocumentQueryTool(BaseTool):
    name = "Document query tool"
    description = "use this tool when you need to get documents for questions related to New Ocean and NOIS"
    
    def _run(self, query: str):
        doc = retriever_public.get_relevant_documents(query)[:3]
        content = reduce(lambda x, y: x + y.page_content, doc, doc[0].page_content)
        
        return content
    
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

        
prefix = """You are an intelligent assistant helping New Ocean and NOIS employees as well as ordinary users with company related questions. You will be provided with a list of keywords that you may use for querying documents.
If you cannot answer using the queried documents, say that you don't know.
You have access to the following tools:"""
suffix = """Begin!"

Keywords: {keywords}
Question: {input}
{agent_scratchpad}"""

tools = [DocumentQueryTool()]

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["keywords", "input", "agent_scratchpad"],
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [14]:
query = "Was Pepsico a customer of NOIS?"
keywords = keyword_getter(query)['text']
keywords_lst = output_parser.parse(keywords)
print(keywords)
print(keywords_lst)

agent_chain.run(input=query, keywords=keywords)

 Pepsico, Pepsico
['Pepsico', 'Pepsico']


> Entering new  chain...
Thought: I need to find out if Pepsico was a customer of NOIS
Action: Document query tool
Action Input: Pepsico
Observation: ��n sự hài lòng cho chúng tôi! Các bạn thực sự là những người cộng sự nhiệt huyết, cảm ơn rất nhiều!
Suntory Pepsico
Trải qua một thời gian hợp tác khá dài, New Ocean là một đối tác dày dạn kinh nghiệm và tận lực trong quá trình hỗ trợ triển khai dự án. Chúng tôi đánh giá cao nỗ lực của các bạn!
Đối tác kinh doanh
Với quá trình gần 20 năm miệt mài nghiên cứu về các dự án giúp t
��n sự hài lòng cho chúng tôi! Các bạn thực sự là những người cộng sự nhiệt huyết, cảm ơn rất nhiều!
Suntory Pepsico
Trải qua một thời gian hợp tác khá dài, New Ocean là một đối tác dày dạn kinh nghiệm và tận lực trong quá trình hỗ trợ triển khai dự án. Chúng tôi đánh giá cao nỗ lực của các bạn!
Đối tác kinh doanh
Với quá trình gần 20 năm miệt mài nghiên cứu về các dự án giúp t
 10% reduction in power consumption, this is 

'Yes, Pepsico was a customer of NOIS.'